In [1]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import models_vit

In [6]:
from IPython.core.display import display, HTML

# Set to display max output in HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Set display max rows and columns
pd.set_option('display.max_rows', None)  # This command shows all rows
pd.set_option('display.max_columns', None)  # This command shows all columns

# Increase the width of columns to stop them wrapping and to show more data
# pd.set_option('display.max_colwidth', None)  # This command shows full content of each column


In [3]:
meta_df = pd.read_csv('/home/sajib/RETFound_MAE/DDI/ddi_metadata.csv')
print(meta_df.shape)
meta_df.head(5)

(656, 6)


,Unnamed: 0,DDI_ID,DDI_file,skin_tone,malignant,disease
0,0,1,000001.png,56,True,melanoma-in-situ
1,1,2,000002.png,56,True,melanoma-in-situ
2,2,3,000003.png,56,True,mycosis-fungoides
3,3,4,000004.png,56,True,squamous-cell-carcinoma-in-situ
4,4,5,000005.png,12,True,basal-cell-carcinoma


In [9]:
meta_df.skin_tone.value_counts()

34    241
12    208
56    207
Name: skin_tone, dtype: int64

In [8]:
meta_df.disease.unique()

array(['melanoma-in-situ', 'mycosis-fungoides',
       'squamous-cell-carcinoma-in-situ', 'basal-cell-carcinoma',
       'squamous-cell-carcinoma', 'melanoma-acral-lentiginous',
       'basal-cell-carcinoma-superficial',
       'squamous-cell-carcinoma-keratoacanthoma',
       'subcutaneous-t-cell-lymphoma', 'melanocytic-nevi',
       'seborrheic-keratosis-irritated', 'focal-acral-hyperkeratosis',
       'hyperpigmentation', 'lipoma', 'foreign-body-granuloma',
       'blue-nevus', 'verruca-vulgaris', 'acrochordon', 'wart',
       'abrasions-ulcerations-and-physical-injuries',
       'basal-cell-carcinoma-nodular', 'epidermal-cyst',
       'acquired-digital-fibrokeratoma', 'epidermal-nevus',
       'seborrheic-keratosis', 'trichilemmoma', 'pyogenic-granuloma',
       'neurofibroma', 'syringocystadenoma-papilliferum',
       'nevus-lipomatosus-superficialis', 'benign-keratosis',
       'inverted-follicular-keratosis', 'onychomycosis', 'dermatofibroma',
       'trichofolliculoma', 'lympho

In [2]:

imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])


def prepare_model(chkpt_dir, arch='vit_large_patch16'):
    # build model
    model = models_vit.__dict__[arch](
        img_size=224,
        num_classes=5,
        drop_path_rate=0,
        global_pool=True,
    )
    # load model
    checkpoint = torch.load(chkpt_dir, map_location='cpu')
    msg = model.load_state_dict(checkpoint['model'], strict=False)
    return model

def run_one_image(img, model):
    
    x = torch.tensor(img)
    x = x.unsqueeze(dim=0)
    x = torch.einsum('nhwc->nchw', x)
    
    x = x.to(device, non_blocking=True)
    latent = model.forward_features(x.float())
    latent = torch.squeeze(latent)
    
    return latent



### Load a pre-trained model

In [3]:
# download pre-trained RETFound 

chkpt_dir = './RETFound_cfp_weights.pth'
model_ = prepare_model(chkpt_dir, 'vit_large_patch16')

device = torch.device('cuda')
model_.to(device)
print('Model loaded.')


Model loaded.


### Load images and save latent feature

In [4]:
# get image list
data_path = './img'
img_list = os.listdir(data_path)

name_list = []
feature_list = []
model_.eval()

for i in img_list:
    img = Image.open(os.path.join(data_path, i))
    img = img.resize((224, 224))
    img = np.array(img) / 255.

    assert img.shape == (224, 224, 3)

    # normalize by mean and sd
    # can use customised mean and sd for your data
    img = img - imagenet_mean
    img = img / imagenet_std
    
    latent_feature = run_one_image(img, model_)
    
    name_list.append(i)
    feature_list.append(latent_feature.detach().cpu().numpy())
    

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 15.57 GiB total capacity; 1.38 GiB already allocated; 1.62 MiB free; 1.42 GiB reserved in total by PyTorch)

In [ ]:
latent_csv = pd.DataFrame({'Name':name_list, 'Latent_feature':feature_list})
latent_csv.to_csv('Feature_latent.csv', index = False, encoding='utf8')

In [ ]:
import torch
torch.cuda.empty_cache()